## Install dependencies

In [ ]:
!pip install -q ipywidgets
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai>=1.8.0"

In [ ]:
from google import genai
from google.genai import types

genai.__version__

## Setup API Key

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

## Automated retry

In [ ]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Required Prompts

- Used few-shot prompt
- Generate response in specific format i.e. python list []

In [ ]:
INGREDIENT_PROMPT = """
Look at this food image and identify all ingredients visible.
Return ONLY a Python list of strings containing the ingredients.
Format your response as a valid Python list, like: ["ingredient1", "ingredient2", "ingredient3"]
Do not include any explanations, headers, or additional text.
"""

RECIPE_MAKING_PROMPT = """
You are a chef. Your task is to help customer design a food recipe based on the ingredients available.
You are allowed to search online using your tool available to find some reference.
Your output should be a markdown format.


Example of Output 1:
-----
## Chicken Stir-Fry with Vegetables
A vibrant and flavorful stir-fry featuring tender slices of chicken, sautéed with crisp vegetables like carrots, bell peppers, and broccoli.
The dish is coated in a savory sauce made from soy sauce, oyster sauce, and a touch of water, giving it a rich, umami flavor.
Quick to prepare and perfect for a healthy, satisfying meal, this dish is best served over steamed rice for a complete dinner.

### Step-by-Step Instructions:

#### Step 1: Marinate the Chicken
- In a bowl, mix the sliced chicken with 1 tablespoon soy sauce and 1 tablespoon cornstarch.
- Let it sit for 10-15 minutes while you prep the veggies.

#### Step 2: Prepare the Sauce
- In a small bowl, mix 1 tablespoon soy sauce, 1 tablespoon oyster sauce, and 1/4 cup water. Set aside.

#### Step 3: Heat the Pan
- Heat 1 tablespoon of oil in a large pan or wok over medium-high heat.

#### Step 4: Cook the Chicken
- Add the marinated chicken to the pan. Cook until it's no longer pink (about 4-5 minutes). Remove and set aside.

#### Step 5: Cook the Vegetables
- Add the remaining oil. Sauté the garlic and onions for 1 minute.
- Add the carrots, bell peppers, and broccoli. Stir-fry for 3–5 minutes until tender-crisp.

#### Step 6: Combine Everything
- Return the chicken to the pan. Pour in the sauce.
- Stir everything together and let it simmer for 2-3 minutes until the sauce thickens slightly.

#### Step 7: Final Touch
- Taste and add salt or pepper if needed.

#### Step 8: Serve
- Serve hot over steamed rice. Enjoy!
-----

Example of Output 2:
-----
## Beef Tacos
A delicious Mexican-inspired dish featuring seasoned ground beef, wrapped in soft or crunchy taco shells and topped with fresh ingredients like lettuce, tomatoes, cheese, and sour cream.
These beef tacos are a fun and customizable meal perfect for family gatherings or a casual dinner.

### Step-by-Step Instructions:

#### Step 1: Cook the Beef
- Heat 1 tablespoon of oil in a skillet over medium heat.
- Add 1 lb ground beef and cook, breaking it up with a spoon, until browned (about 5-7 minutes).
  
#### Step 2: Season the Beef
- Stir in 1 tablespoon taco seasoning, 1/2 cup water, and simmer for 3–5 minutes until the mixture thickens.
  
#### Step 3: Prepare the Toppings
- Chop lettuce, tomatoes, and shred the cheese.
- Prepare any additional toppings like onions, avocado, and cilantro.

#### Step 4: Heat the Taco Shells
- Warm the taco shells in the oven or microwave according to the package instructions.

#### Step 5: Assemble the Tacos
- Spoon the beef mixture into the taco shells.
- Add the chopped lettuce, tomatoes, cheese, and any additional toppings you like.

#### Step 6: Serve
- Serve immediately with a side of salsa and sour cream. Enjoy!
-----

Example of Output 3:
-----
## Spaghetti Carbonara
A classic Italian pasta dish made with creamy eggs, crispy pancetta, and parmesan cheese, mixed with spaghetti for a rich, comforting meal.
The simplicity of this dish makes it a favorite for both beginners and experienced cooks.

### Step-by-Step Instructions:

#### Step 1: Cook the Spaghetti
- Bring a large pot of salted water to a boil and cook 12 oz spaghetti according to package instructions. Drain, reserving 1/2 cup pasta water.

#### Step 2: Prepare the Sauce
- In a bowl, whisk together 2 eggs, 1/2 cup grated parmesan, and a pinch of black pepper. Set aside.

#### Step 3: Cook the Pancetta
- In a skillet, cook 4 oz pancetta or bacon over medium heat until crispy (about 4-5 minutes). Remove from heat.

#### Step 4: Combine Pasta and Pancetta
- Add the cooked spaghetti to the skillet with the pancetta. Toss well to combine, letting the heat of the pasta warm the pancetta.

#### Step 5: Make the Carbonara Sauce
- Slowly pour the egg and cheese mixture into the pasta, tossing quickly to avoid scrambling the eggs. Add a bit of reserved pasta water to create a creamy texture.

#### Step 6: Serve
- Serve immediately with extra parmesan and a sprinkle of black pepper. Enjoy!
-----

Ingredient List: {}


Based on all the ingredients above, here is my idea!
##
"""

## File upload widget & image processing function

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Food Image:',
    layout=widgets.Layout(width='300px')
)
display(uploader)

def process_uploaded_image(uploader):
    if not uploader.value:
        print("No image uploaded.")
        return None

    uploaded_file = uploader.value[0]
    image_data = uploaded_file["content"]
    return image_data

## Determine ingredients from image & generate recipe using LLM

- Utilised image understanding

In [ ]:
import tempfile
import os

client = genai.Client(api_key=GOOGLE_API_KEY)

def upload_image_to_genai(image_bytes):
    
    # Create a temporary file to save the image bytes
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_file:
        temp_file.write(image_bytes)
        temp_file_path = temp_file.name
    
    try:
        uploaded_file = client.files.upload(file=temp_file_path)
        return uploaded_file
    finally:
        os.remove(temp_file_path)


def get_ingredients(image_bytes):
    uploaded_image = upload_image_to_genai(image_bytes)
    
    # Generate ingredients list
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[
            uploaded_image,
            "\n\n",
            INGREDIENT_PROMPT
        ],
        config=types.GenerateContentConfig(temperature=0.0)
    )
    
    text = response.text
    
    # If response is in a code block, extract just the content
    if "```" in text:
        # Extract content between backticks
        import re
        code_content = re.search(r'```(?:python)?\n(.*?)\n```', text, re.DOTALL)
        if code_content:
            text = code_content.group(1).strip()
    
    try:
        ingredients_list = eval(text)
        return ingredients_list
    except:
        print("Warning: Could not parse the response as a list. Using raw text.")
        return text


def generate_recipe(ingredients_list):
    ingredients_str = ", ".join(ingredients_list)
    
    prompt = RECIPE_MAKING_PROMPT.replace("{}", ingredients_str)
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config=types.GenerateContentConfig(temperature=0.7)
    )
    
    return response.text



## Run code to get ingredients list & generate a recipe based on it

In [ ]:
image_bytes = process_uploaded_image(uploader)

if image_bytes:
    ingredients = get_ingredients(image_bytes)
    print("Ingredients identified:\n")
    
    if isinstance(ingredients, list):
        for ingredient in ingredients:
            print(f"- {ingredient}")
            
        # Generate recipe
        recipe = generate_recipe(ingredients)
        print("\nGENERATED RECIPE:\n")
        print(recipe)
    else:
        print(ingredients)